# Getting Started with the Gemini Sandbox API
## ... with `ccxt`


### Before you'll be able to run this notebook

### Create a Gemini Sandbox Account 
Your coinbase account can be used in a sandbox mode

* Sign up to a `sandbox` Gemini account: https://exchange.sandbox.gemini.com/register
* Login to https://exchange.sandbox.gemini.com/
* Go to the API section and generate an API key 
    * Go to Account -> Settings -> API  
    * IMPORTANT! Create a `Primary Key` and NOT `Master Key`. 
* Make sure to save the API Secret, and API Key in a safe place
* Save your API keys in you `bash_profile`


_Note: The Gemini sandbox account comes with enough fake USD, BTC, and ETH to get you started._

#### NB Setup commands

In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from IPython.display import HTML


def set_paths() -> None:

    if "cwd" not in globals():
        global cwd
        cwd = Path(os.getcwd()).parents[1]
        os.chdir(cwd)


set_paths()
display(HTML("<style>.container { width:80% !important; }</style>"))

<IPython.core.display.Javascript object>

In [2]:
# All modules within the src/crypto can be called like this:
from crypto.utils import config_utils
import ccxt

import pandas as pd

<IPython.core.display.Javascript object>

In [ ]:
# Create an instance

In [3]:
exchange = ccxt.gemini(
    {
        "apiKey": os.environ.get("GEMINI_SNDBX_API_KEY"),
        "secret": os.environ.get("GEMINI_SNDBX_API_SECRET"),
    }
)

<IPython.core.display.Javascript object>

In [4]:
# This has to be set before any
exchange.set_sandbox_mode(True)

<IPython.core.display.Javascript object>

In [26]:
# If your API key, secret or passwords is not setup correctly,
# ... you'll get an authentication error
account_balance = exchange.fetch_balance()
pd.DataFrame.from_records(account_balance["info"]).dtypes

type                      object
currency                  object
amount                    object
available                 object
availableForWithdrawal    object
dtype: object

<IPython.core.display.Javascript object>

# Getting historical data

OHLCV is an aggregated form of market data standing for Open, High, Low, Close and Volume. OHLCV data includes 5 data points: the Open and Close represent the first and the last price level during a specified interval. High and Low represent the highest and lowest reached price during that interval. Volume is the total amount traded during that period. This data is most frequently represented in a candlestick chart, which allows traders to perform technical analysis on intraday values. We provide OHLCV data in granularities ranging from 1 second to 1 day. 



* Timestamp	Epoch timestamp in milliseconds. You can learn more about timestamps, including how to convert them to human readable form, here. 
* Open	Opening price of the time interval in quote currency (For BTC/USD, the price would be USD).
* High	Highest price reached during time interval, in quote currency.
*Low	Lowest price reached during time interval, in quote currency.
* Close	Closing price of the time interval, in the quote currency. 
* Volume	Quantity of asset bought or sold, displayed in base currency.

In [ ]:
# Available timeframes
exchange.timeframes

In [6]:
from_datetime = "2021-10-22 00:00:00"
from_timestamp = exchange.parse8601(from_datetime)
from_timestamp

1634860800000

<IPython.core.display.Javascript object>

In [7]:
ohlcvs = exchange.fetch_ohlcv(
    symbol="BTC/USD", timeframe="5m", since=from_timestamp, limit=None
)

<IPython.core.display.Javascript object>

In [8]:
df_historical_data = pd.DataFrame.from_records(
    ohlcvs, columns=["timestamp", "open", "high", "low", "close", "volume"]
).assign(datetime=lambda x: pd.to_datetime(x["timestamp"], unit="ms"))

<IPython.core.display.Javascript object>

# Start trading!

## Buy

In [11]:
exchange.create_limit_sell_order("BTC/USD", amount=900, price="47071")

{'info': {'order_id': '1330836961',
  'id': '1330836961',
  'symbol': 'btcusd',
  'exchange': 'gemini',
  'avg_execution_price': '0.00',
  'side': 'sell',
  'type': 'exchange limit',
  'timestamp': '1639841889',
  'timestampms': '1639841889023',
  'is_live': True,
  'is_cancelled': False,
  'is_hidden': False,
  'was_forced': False,
  'executed_amount': '0',
  'client_order_id': '1639841888698',
  'options': [],
  'price': '47071.00',
  'original_amount': '900',
  'remaining_amount': '900'},
 'id': '1330836961'}

<IPython.core.display.Javascript object>

In [ ]:
# Check balance
account_balance = exchange.fetch_balance()
pd.DataFrame.from_records(account_balance["info"])

In [ ]:
exchange.create_market_buy_order()

## Sell! 

In [ ]:
exchange.create_market_sell_order("BTC/USD", amount=1)

In [ ]:
# Check balance
account_balance = exchange.fetch_balance()
pd.DataFrame.from_records(account_balance["info"])

In [ ]:
## Appendix: To Find Exchanges with a Sanbox API

In [ ]:
with_sandbox = []
without_sanbox = []

for exchange_id in ccxt.exchanges:
    exchange = getattr(ccxt, exchange_id)()

    test_urls = exchange.urls.get("test")
    if test_urls:
        with_sandbox.append((exchange, test_urls))
    else:
        without_sanbox.append((exchange, "No Sandbox"))

In [ ]:
ccxt.ftx

In [ ]:
pd.DataFrame.from_records(with_sandbox)